# Tests for Portfolio VaR Functions

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
import plotly.express as px

# Import functions
import PortfolioVaR as pv

### Asset Normal/ Undiversified VaR

**NOTE**: figure out later if backtesting here makes sense, it should be possible.

**NOTE 2**: since we already have the parametric VaR, a Portfolio-Normal VaR function is redundant.

In [2]:
# Set parameters
confidence_level = 0.99
holding_period = 1

In [3]:
# Define a small time series of positions (3 assets over 5 days)
position_data = pd.DataFrame({
    "Asset_A": [98, 102, 104, 100, 108],
    "Asset_B": [167, 198, 154, 145, 206],
    "Asset_C": [345, 303, 306, 287, 312]
}, index=pd.date_range("2023-01-01", periods=5))

In [4]:
# Compute Diversified VaR
var_series = pv.var_asset_normal(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period,
    undiversified=False
)

# Compute Undiversified VaR
uvar_series = pv.var_asset_normal(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period,
    undiversified=True
)

# Compute Diversification Benefit
div_benefit = uvar_series - var_series

# Combine in a DataFrame
result = pd.DataFrame({
    "Diversified_VaR": var_series,
    "Undiversified_VaR": uvar_series,
    "Diversification_Benefit": div_benefit
})

# Print result
print(result)


            Diversified_VaR  Undiversified_VaR  Diversification_Benefit
2023-01-02       169.962282         205.062609                35.100326
2023-01-03       144.368383         177.109070                32.740687
2023-01-04       135.965647         166.762800                30.797153
2023-01-05       176.575235         212.882681                36.307446


In [5]:
# One-row static input
static_position = pd.DataFrame([[1_000_000, 2_000_000, 3_000_000]])

var = pv.var_asset_normal(static_position, confidence_level=0.99)
# -> Warning will be shown, and function returns None

/Users/aledo/Documents/GitHub/VaR/Functions/PortfolioVaR.py:46: UserWarning: You must provide a time series of positions (at least 2 rows).
  warnings.warn("You must provide a time series of positions (at least 2 rows).")


### Marginal VaR

In [6]:
# Call the marginal_var function
marginal_df = pv.marginal_var(
    position_data=position_data,
    confidence_level=confidence_level,
    holding_period=holding_period
)

# Print the result
print(marginal_df)

             Asset_A   Asset_B   Asset_C
2023-01-02  0.094562  0.605458  0.133453
2023-01-03  0.094818  0.583709  0.145804
2023-01-04  0.094896  0.584077  0.145592
2023-01-05  0.094614  0.606175  0.132965


### Incremental, Component and Relative Component VaR

In [7]:
# Test Component VaR
component_df = pv.component_var(position_data, confidence_level, holding_period)
print("Component VaR:")
print(component_df)

# Test Relative Component VaR
relative_df = pv.relative_component_var(position_data, confidence_level, holding_period)
print("\nRelative Component VaR:")
print(relative_df)

# Test Incremental VaR (e.g., add 100 to Asset_B)
change_vector = [15, 100, 50]
ivar_series = pv.incremental_var(position_data, change_vector, confidence_level, holding_period)
print("\nIncremental VaR (100 increase in Asset_B):")
print(ivar_series)

Component VaR:
              Asset_A     Asset_B    Asset_C
2023-01-02   9.645305  119.880720  40.436257
2023-01-03   9.861094   89.891173  44.616116
2023-01-04   9.489565   84.691162  41.784920
2023-01-05  10.218265  124.871977  41.484993

Relative Component VaR:
             Asset_A   Asset_B   Asset_C
2023-01-02  0.056750  0.705337  0.237913
2023-01-03  0.068305  0.622651  0.309044
2023-01-04  0.069794  0.622886  0.307320
2023-01-05  0.057869  0.707189  0.234942

Incremental VaR (100 increase in Asset_B):
2023-01-02    68.636895
2023-01-03    67.083380
2023-01-04    67.110735
2023-01-05    68.684904
Freq: D, dtype: float64


### Final Check

In [8]:
# 1. Get total VaR from your main function
var_series = pv.var_asset_normal(position_data, confidence_level)

# 2. Sum of Component VaR per day
component_df = pv.component_var(position_data, confidence_level)
component_sum = component_df.sum(axis=1)

# 3. Sum of Relative Component VaR per day
relative_df = pv.relative_component_var(position_data, confidence_level)
relative_sum = relative_df.sum(axis=1)

# 4. Build comparison table
check_df = pd.DataFrame({
    "Portfolio_VaR": var_series,
    "Sum_Component_VaR": component_sum,
    "Diff": var_series - component_sum,
    "Sum_Relative_CVaR": relative_sum
})

print(check_df)


            Portfolio_VaR  Sum_Component_VaR          Diff  Sum_Relative_CVaR
2023-01-02     169.962282         169.962282 -2.842171e-14                1.0
2023-01-03     144.368383         144.368383  2.842171e-14                1.0
2023-01-04     135.965647         135.965647 -2.842171e-14                1.0
2023-01-05     176.575235         176.575235  0.000000e+00                1.0


### Visualizations

In [9]:
pv.plot_var_series(var_series, uvar_series)

In [10]:
pv.plot_risk_contribution_pie(component_df)

In [11]:
pv.plot_risk_contribution_lines(component_df)

---
### Appendix

- **Asset-Normal VaR (AN VaR)**  
  $$
  \text{VaR}_t = z_\alpha \cdot \sqrt{x_t^\top \Sigma x_t} \cdot \sqrt{h}
  $$

- **Undiversified VaR (UVaR)**  
  $$
  \text{UVaR}_t = z_\alpha \cdot \sum_{i=1}^N \sigma_i x_{i,t} \cdot \sqrt{h}
  $$

- **Marginal VaR**  
  $$
  \Delta \text{VaR}_{i,t} = \text{VaR}_t \cdot \frac{(\Sigma x_t)_i}{x_t^\top \Sigma x_t}
  $$

- **Component VaR**  
  $$
  \text{CVaR}_{i,t} = x_{i,t} \cdot \Delta \text{VaR}_{i,t}
  $$

- **Relative Component VaR**  
  $$
  \text{RCVaR}_{i,t} = \frac{\text{CVaR}_{i,t}}{\text{VaR}_t}
  $$

- **Incremental VaR**  
  $$
  \text{IVaR}_t = \Delta \text{VaR}_t^\top \cdot a
  $$

---

### Function Dependencies

```text
var_asset_normal()
 └── marginal_var()
      ├── component_var()
      │     └── relative_component_var()
      └── incremental_var()
```

---

### Notes
- All risk measures are in **monetary terms**.
- Covariance is based on returns inferred from time series of holdings.
- Inputs: matrix of monetary holdings, confidence level $z_\alpha$, and horizon $h$.